In [8]:
import sys
import os
# 프로젝트 폴더명 입력
sys.path.insert(0, os.getcwd()+'/drive/My Drive/Colab Notebooks/recommend_music_by_sentiment_analysis')
import config

word_cloud를 위한 한글 글씨체 다운로드

In [ ]:
!apt-get install fonts-nanum*
!apt-get install fontconfig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'fonts-nanum-eco' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-gothic-light' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-coding' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-extra' for glob 'fonts-nanum*'
fonts-nanum is already the newest version (20170925-1).
fonts-nanum-coding is already the newest version (2.5-1).
fonts-nanum-eco is already the newest version (1.000-6).
fonts-nanum-extra is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fontconfig is already the newest version (2.12.6-0ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


wordcloud 생성

In [ ]:
!pip install WordCloud
import sys
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import pandas as pd


# --------------------------------------- 변수 -------------------------------
DATASET_PATH = config.PATH_CONFIG['DATASET_PATH']
RESULT_PATH = config.PATH_CONFIG['RESULT_PATH']
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
EXCEL_NAME = 'angry.xlsx'
PNG_NAME = 'angry.png'
COMMON_NUMBER = 50
# 기쁨(0) 슬픔(1) 우울(2) 설렘(3) 화남(4)
EMOTION_LIST = ["happy", 'sad', 'depressed', 'love', 'angry']
# ----------------------------------------------------------------------------

# 엑셀의 문장들을 하나의 문장으로 합침
def make_docs(): 
  xlsx = pd.read_excel(DATASET_PATH + EXCEL_NAME, names = ['c1', 'c2'])
  docs = ""
  for i in xlsx['c1']:
    docs+=str(i)+'.'
  return docs

# 불용어 사전 구축
def make_stop_words():
  f = open(DATASET_PATH + 'stop_words_kr.txt','r')
  stop_words = []
  while True:
      line = f.readline()
      if not line: break
      stop_words.append(line[:-1])
  return stop_words

# 고빈도 명사 리스트 구축
def get_noun(news, stop_words):
  okt = Okt()
  noun_ = okt.nouns(news)
  noun = []
  for i in noun_:
    noun.append(i.replace(" ", ""))
  new_noun = []
  for i, v in enumerate(noun):
    if (len(v) >= 2) and (v not in stop_words):
      new_noun.append(v)
      
  count = Counter(new_noun)
  noun_list = count.most_common(COMMON_NUMBER)
  return noun_list

# wordcloud 생성
def visualize(noun_list):
  wc = WordCloud(font_path = FONT_PATH,
                 background_color = "white",
                 width = 1000,
                 height = 1000,
                 max_words = COMMON_NUMBER,
                 max_font_size = 300)
  wc.generate_from_frequencies(dict(noun_list))
  wc.to_file(RESULT_PATH + PNG_NAME)

for emotion in EMOTION_LIST:
  EXCEL_NAME = emotion + '.xlsx'
  PNG_NAME = emotion + '.png'
  docs = make_docs()
  stop_words = make_stop_words()
  noun_list = get_noun(docs, stop_words)
  visualize(noun_list)
  print('finish ' + emotion)

finish happy
finish sad
finish depressed
finish love
finish angry
